In [1]:
using JLD
files = readdir("saves")

all_phi1_full_orig = Array(Any, 100)
all_phi1_full_log = Array(Any, 100)
all_phi1_red_orig = Array(Any, 100)
all_phi1_red_log = Array(Any, 100)

phi0_red_orig = []
phi0_red_log = []
phi0_full = [2.2e-6, 0.42, 0.1026, 0.58, 39*1e-6, 6.16,30.4,2.0];

n = 100
for i = 1:n
    phi1_red_orig, phi1_red_log, fname = load("saves/figure_A2_hess_"*string(i)*".jld", "phi1_orig", "phi1_log", "A1_filename")
    phi1_full_orig, phi1_full_log = load("saves/"*fname, "phi1_orig", "phi1_log")
    
    if i == 1
        phi0_red_orig, phi0_red_log = load("saves/figure_A2_hess_"*string(i)*".jld", "phi0_orig", "phi0_log")
    end
        
    all_phi1_full_orig[i] = phi1_full_orig
    all_phi1_full_log[i] = phi1_full_log
    all_phi1_red_orig[i] = phi1_red_orig
    all_phi1_red_log[i] = phi1_red_log
end

In [2]:
all_phi1_full_orig = hcat(all_phi1_full_orig[1:n]...)
all_phi1_full_log = hcat(all_phi1_full_log[1:n]...)
all_phi1_red_orig = hcat(all_phi1_red_orig[1:n]...)
all_phi1_red_log = hcat(all_phi1_red_log[1:n]...)

6x100 Array{Float64,2}:
 3.20898e-8   1.35829e-8   2.46993e-8  …  1.55956e-7  1.84893e-7   5.50225e-9
 8.94143     12.5268      11.2877         5.94941     5.55527     16.0566    
 0.566465     0.517835     0.50775        0.553083    0.569307     0.498821  
 3.21388e-5   3.22868e-5   3.98173e-5     3.5816e-5   3.15686e-5   3.47983e-5
 8.95066      9.76507      4.22656        4.80449     5.05053      7.87794   
 1.12244      0.894068     2.33578     …  2.38582     2.15446      1.20695   

In [3]:
std(all_phi1_full_orig,2)./phi0_full

8x1 Array{Float64,2}:
 52710.2       
     1.23427   
     4.25743   
     0.269498  
     2.44372e23
     1.59875e9 
     1.02409e10
     2.30359e23

In [4]:
std(all_phi1_full_log,2)./phi0_full

8x1 Array{Float64,2}:
 52.6923  
  1.3687  
  0.884629
  0.510915
  0.164124
  0.285845
  1.48736 
  0.358432

In [5]:
std(all_phi1_red_orig,2)./phi0_red_orig

5x1 Array{Float64,2}:
 0.210585 
 0.0459081
 0.0953939
 0.106912 
 0.234979 

In [18]:
sqrt(mean(broadcast(./, broadcast(-, all_phi1_red_orig, phi0_red_orig), phi0_red_orig).^2, 2))

5x1 Array{Float64,2}:
 0.212376 
 0.0456782
 0.0952892
 0.107906 
 0.245332 

In [21]:
std(broadcast(./, broadcast(-, all_phi1_red_orig, phi0_red_orig), phi0_red_orig),2)

5x1 Array{Float64,2}:
 0.210585 
 0.0459081
 0.0953939
 0.106912 
 0.234979 

In [16]:
broadcast(./, broadcast(-, all_phi1_full_orig, phi0_full), phi0_full)

8x100 Array{Float64,2}:
 -0.985401    1.10128    16.6542    …  11.3875     6.76939   -0.362724
 -0.612061   -0.0289361   1.31088      -0.72256   -0.211466   1.0025  
 -0.407614    0.0276091   0.669403     -0.554278  -0.375534   0.828934
  0.0973187   0.125674    0.300148      0.199032   0.128733   0.337758
  0.355865   -0.0817109   0.136259      0.111821   0.292374  -0.168711
 -0.5889      0.401309    0.625671  …  -0.501361  -0.440348   2.33092 
  3.45901    -0.277929   -0.819214     -0.128914  -0.283813  -0.531539
  2.99946    -0.365774   -0.305584      1.85203    1.91727   -0.776723

In [8]:
?broadcast!

search: broadcast! broadcast!_function broadcast_setindex! broadcast



```
broadcast!(f, dest, As...)
```

Like `broadcast`, but store the result of `broadcast(f, As...)` in the `dest` array. Note that `dest` is only used to store the result, and does not supply arguments to `f` unless it is also listed in the `As`, as in `broadcast!(f, A, A, B)` to perform `A[:] = broadcast(f, A, B)`.


In [6]:
std(all_phi1_red_log,2)./phi0_red_log

6x1 Array{Float64,2}:
 5.12874  
 0.546935 
 0.0481257
 0.132807 
 0.480718 
 0.486401 